This notebook aims to add drug_id and strtify the training set. Most of the code here is copied from @cdeotte and @gogo827jz available in this [discussion](https://www.kaggle.com/c/lish-moa/discussion/195195)

In [ ]:
import pandas as pd
import numpy as np
import sys
import gc
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def create_folds(seed_count, fold_count):

    folds = []

    train_features = pd.read_csv('../input/lish-moa/train_features.csv')
    train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
    train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')


    # Get rid of "ctl_vehicle" from training. 
    # You may comment below lines if you do not want to do it.
    train_targets_scored = train_targets_scored.loc[train_features['cp_type'] == 'trt_cp', :]
    train_features = train_features[train_features['cp_type'] == 'trt_cp']
    
    train_features_drug = train_features.merge(train_drug, on="sig_id", how='left')
    
    # Add drug_id as one of the targets (for stratifying later)
    targets = train_targets_scored.columns[1:]
    train_targets_scored = train_targets_scored.merge(train_drug, on='sig_id', how='left') 

    # Within in training data, identify indices where drug ids 
    # which are present in more than 18 rows and less than 18 rows 
    vc = train_targets_scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index
    vc2 = vc.loc[vc > 18].index

    # tmp is a dataframe derived from scored targets, where targets are 
    # averaged by drugid (one row per drug id)
    tmp = train_targets_scored.groupby('drug_id')[targets].mean().loc[vc1]
    tmp = tmp.reset_index()    
    tmp = tmp.rename(columns={"index":"drug_id"})
    
    # tmp1 is a dataframe with tagets and drug_id for all drugs that have 
    # repeated more that 18 times in train dataset.
    # We are stratifying these drugs as among all folds. 
    # Thought here is that such drugs might repeat in public/private test sets as well
    tmp1 = train_targets_scored[train_targets_scored['drug_id'].isin(vc2)]
    tmp1 = tmp1.reset_index(drop=True)

    for seed in range(seed_count):

        skf = MultilabelStratifiedKFold(n_splits = fold_count, shuffle = True, random_state = seed)
        tmp_copy = tmp.copy()
        tmp1_copy = tmp1.copy()
        train_indices = train_features_drug[['sig_id', 'drug_id']].copy()
        
        for fold,(idxT,idxV) in enumerate(skf.split(X=tmp_copy,y=tmp_copy[targets])):
            tmp_copy.loc[idxV,"kfold"] = fold
        train_indices = train_indices.merge(tmp_copy[['drug_id', 'kfold']], on='drug_id', how="left")

        for fold,(idxT,idxV) in enumerate(skf.split(X=tmp1_copy,y=tmp1_copy[targets])):
            tmp1_copy.loc[idxV,"kfold"] = fold        
        train_indices = train_indices.merge(tmp1_copy[['sig_id', 'kfold']], on='sig_id', how="left")

        train_indices['kfold'] = train_indices['kfold_x'].combine_first(train_indices['kfold_y'])        
        train_indices.drop(['drug_id', 'kfold_x', 'kfold_y'], inplace=True, axis=1) 
        
        # Add this to the output
        folds.append(train_indices)       

    return np.stack(folds)

In [ ]:
# Four seeds, 5 folds
folds = create_folds(4, 5)

In [ ]:
folds

In [ ]:
# First dimention indicates number of seeds
folds.shape